# Pruning ResNet50 Model
This notebook shows how to reduce the size of a model by pruning its parameters. It assumes 
that a trained ```ResNet50``` model already exists in the ```Models``` directory. Please refer to the notebook
[Image Classification with ResNet50](ResNet50.ipynb) for more info about using a pretrained ResNet50 model.

If you want to prune a Low-Rank model, you can use [this](ResNet50-Reduce.ipynb) notebook
to reduce the number of parameters in ```ResNet50```.

## Load and evaluate the original pretrained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus='upto4'

# Create the test dataset for evaluation.
testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Cafe', numWorkers=8)

orgFileName = "Models/ResNet50RR.fbm"    # Reduced - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.initSession()
model.printLayersInfo()
results = model.evaluate(topK=5)



Reading from "Models/ResNet50RR.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      9,472      
S1_L2_BN         112 112 64                             56 56 64      ReLU     MP(KSP):3 2 1    256        
S2_L1_RES2       56 56 64      2 Paths, 8 layers        56 56 256     ReLU                      76,928     
S2_L2_RES1       56 56 256     2 Paths, 7 layers        56 56 256     ReLU                      71,552     
S2_L3_RES1       56 56 256     2 Paths, 7 layers        56 56 256     ReLU                      71,552     
S3_L1_RES2c1     56 56 256     2 Paths

## Pruning the model
Here we prune the model using the [pruneModel](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.pruneModel) class method of the model.

In [2]:
prunedFileName = orgFileName.replace('.fbm', 'P.fbm')  # Append 'P' to the filename for "Pruned"
pResults = Model.pruneModel(orgFileName, prunedFileName, mseUb=.00001)


Reading model parameters from "Models/ResNet50RR.fbm" ... Done.
Pruning 363 tensors using 36 workers ... 
   Pruning Parameters:
        mseUb ................ 0.000010
Pruning process complete (3.83 Sec.)
Now saving to "Models/ResNet50RRP.fbm" ... Done.

Number of parameters: 11,680,232 -> 7,896,379 (3,783,853 pruned)
Model File Size: 46,745,545 -> 33,056,590 bytes


## Evaluate the pruned model
Compare the new number of parameters with the original. Let's see the impact of this reduction to the performance of the model.

In [3]:
model = Model.makeFromFile(prunedFileName, testDs=testDs, gpus='0')   
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/ResNet50RRP.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      7,114      
S1_L2_BN         112 112 64                             56 56 64      ReLU     MP(KSP):3 2 1    256        
S2_L1_RES2       56 56 64      2 Paths, 8 layers        56 56 256     ReLU                      48,512     
S2_L2_RES1       56 56 256     2 Paths, 7 layers        56 56 256     ReLU                      48,277     
S2_L3_RES1       56 56 256     2 Paths, 7 layers        56 56 256     ReLU                      48,808     
S3_L1_RES2c1     56 56 256     2 Path

## Re-training after pruning
Here we first make a new model from the file created above. We then call the [train](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.train) method of the model to start the re-training.

After re-training, we run the [evaluate](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.evaluate) function again to see how the re-training improved the performance
of the model.

The re-trained model is then saved to a file appending an 'R' letter (for Re-trained) to the end of the pruned model file name.

In [4]:
tuneDs = ImageNetDSet.makeDatasets('tune', batchSize=256, preProcessing='Crop256Cafe', numWorkers=8)
print(tuneDs)

model = Model.makeFromFile(prunedFileName, trainDs=tuneDs, testDs=testDs,
                           numEpochs=2,
                           learningRate=(0.00005, 0.00001),
                           optimizer="Adam",
                           gpus=gpus)
model.printLayersInfo()
model.initSession()
model.train()
results = model.evaluate(topK=5)

retrainedFileName = prunedFileName.replace('.fbm', 'R.fbm')  # Append 'R' to the filename for "Retrained"
model.save(retrainedFileName)

ImageNetDSet Dataset Info:
    Dataset Name ................................... tune
    Dataset Location ............................... /data/ImageNet/
    Number of Classes .............................. 1000
    Number of Samples .............................. 64000
    Sample Shape ................................... (224, 224, 3)
    Preprocessing .................................. Crop256Cafe
    Number of Workers .............................. 8


Reading from "Models/ResNet50RRP.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      7,114      
S1_L2_BN  

## Also look at

[Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)

[Exporting ResNet50 Model to ONNX](ResNet50-ONNX.ipynb)

[Exporting ResNet50 Model to TensorFlow](ResNet50-TF.ipynb)

[Exporting ResNet50 Model to CoreML](ResNet50-CoreML.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with ResNet50](ResNet50.ipynb)

[Reducing number of parameters of ResNet50 Model](ResNet50-Reduce.ipynb)